In [1]:
import cv2
import easyocr
from ultralytics import YOLO

In [2]:
# Step 1: Load YOLO model for license plate detection
model = YOLO("D:/anpr2/runs/detect/train_final.pt")  # Load your trained YOLO model

In [3]:
# Step 2: Initialize EasyOCR Reader for text recognition (OCR)
reader = easyocr.Reader(['en'], gpu=True)  # Initialize OCR reader with English

In [4]:
# Step 3: Load an image for inference
img_path = "D:/anpr2/image3.jpg"  # Path to your image
img = cv2.imread(img_path)

In [5]:
# Step 4: Perform inference to detect license plate
results = model.predict(img)


0: 384x640 1 license_plate, 75.0ms
Speed: 6.0ms preprocess, 75.0ms inference, 153.7ms postprocess per image at shape (1, 3, 384, 640)


In [6]:
for result in results:
    boxes = result.boxes.xyxy  # Coordinates for the bounding boxes
    if len(boxes) > 0:
        # Take the first detected bounding box (for the license plate)
        box = boxes[0]
        x1, y1, x2, y2 = map(int, box)  # Convert to integer

        # Step 6: Crop the license plate from the image
        cropped_license_plate = img[y1:y2, x1:x2]

        # Step 7: Preprocess the cropped license plate for OCR
        gray = cv2.cvtColor(cropped_license_plate, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Threshold
        denoised = cv2.GaussianBlur(thresh, (5, 5), 0)  # Denoising

        # Step 8: Perform OCR (text recognition) on the cropped and preprocessed license plate
        result = reader.readtext(denoised)

        # Step 9: Extract and display the detected text (license plate number)
        for detection in result:
            license_plate_text = detection[1]  # Extracted text
            print(f"Detected License Plate: {license_plate_text}")
            
            # Step 10: Draw the bounding box and recognized text on the image
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, license_plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


Detected License Plate: (BJY 982]


In [8]:
  # Show the result image
cv2.imshow("Detected License Plate", img)
cv2.waitKey(0)
cv2.destroyAllWindows()